# Hyperparameter Tuning using HyperDrive

In [22]:
from azureml.core import Workspace, Environment, Experiment, Model
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import BayesianParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice
from azureml.train.hyperdrive.parameter_expressions import uniform
import os
import joblib
import pandas as pd
ws = Workspace.get(name="quick-starts-ws-136065")
exp = Experiment(workspace=ws, name="lol_match_predictor")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-136065
Azure region: southcentralus
Subscription id: 2c48c51c-bd47-40d4-abbe-fb8eabd19c8c
Resource group: aml-quickstarts-136065


In [5]:
# Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

cluster_name = "cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS3_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

Creating
Succeeded..

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [10]:
ws = Workspace.from_config()
experiment_name = 'lol_match_predictor'

experiment=Experiment(ws, experiment_name)

In [101]:
dataset = pd.read_csv("https://raw.githubusercontent.com/ncharchenko/Azure-ML-Engineer-Nanodegree-Capstone/master/OraclesElixir_lol_match_data_teams.csv")

# Show raw data before it is processed.
dataset.head()

,side,result,firstblood,firstdragon,firstherald,heralds,firsttower,firstmidtower,golddiffat10,xpdiffat10,csdiffat10,killsat10,deathsat10,golddiffat15,xpdiffat15,csdiffat15,killsat15,deathsat15
0,Blue,1,0,0.0,1.0,1.0,0.0,0.0,117.0,-137.0,8.0,0.0,0.0,-825.0,-1665.0,-12.0,0.0,1.0
1,Red,0,1,1.0,0.0,1.0,1.0,1.0,-117.0,137.0,-8.0,0.0,0.0,825.0,1665.0,12.0,1.0,0.0
2,Blue,0,0,0.0,0.0,0.0,0.0,1.0,-2014.0,-2149.0,-34.0,0.0,3.0,-5484.0,-4833.0,-56.0,0.0,6.0
3,Red,1,1,1.0,1.0,2.0,1.0,0.0,2014.0,2149.0,34.0,3.0,0.0,5484.0,4833.0,56.0,6.0,0.0
4,Blue,1,0,1.0,1.0,2.0,1.0,1.0,682.0,615.0,25.0,0.0,0.0,2566.0,2097.0,53.0,0.0,0.0


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [17]:
#TODO: Create the different params that you will be using during training
param_sampling = BayesianParameterSampling( {
        "--C": uniform(0.1,10),
        "--max_iter": choice(20,40,60,80,100,150,200,300,400,600,800)
    }
)

if "training" not in os.listdir():
    os.mkdir("./training")

#TODO: Create your estimator and hyperdrive config
estimator = SKLearn(".", compute_target=compute_target, entry_script="train.py")

hyperdrive_run_config = HyperDriveConfig(param_sampling,
                                     estimator=estimator,
                                     primary_metric_name="Accuracy",
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=100,
                                     max_concurrent_runs=4)






'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [18]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = exp.submit(hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [19]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [54]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments'])

['--C', '5.459884368694646', '--max_iter', '100']


In [85]:
#TODO: Save the best model

joblib.dump(value=best_run.id, filename="model.pkl")

# Register best model
model = Model.register(model_path = "model.pkl",
                       model_name = "lol-predictor",
                       description = "League of Legends match predictor trained using Azure ML.",
                       workspace = ws)

Registering model lol-predictor


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [91]:
# Get environment
env = Environment.get(workspace=ws, name="AzureML-AutoML")

# Create inference configuration
inference_config = InferenceConfig(entry_script="score.py", environment=env)

# Set deployment configuration as an Azure Container Instance. Note logging is enabled
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=4, enable_app_insights=True)

# Service deployment
service = Model.deploy(
    workspace=ws,
    name="lolpredictor",
    models=[model],
    inference_config=inference_config,
    deployment_config=deployment_config
)

service.wait_for_deployment(show_output=True)
print(service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running...............................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [90]:
print(service.get_logs())

2021-01-26T05:37:49,381704583+00:00 - iot-server/run 
2021-01-26T05:37:49,382781013+00:00 - rsyslog/run 
2021-01-26T05:37:49,384896873+00:00 - gunicorn/run 
/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
2021-01-26T05:37:49,386768027+00:00 - nginx/run 
rsyslogd

TODO: In the cell below, send a request to the web service you deployed to test it.

In [92]:
scoring_uri = service.scoring_uri

print("Service State: {}".format(service.state))
print("Scoring URI: {}".format(service.scoring_uri))
print("Swagger URI: {}".format(service.swagger_uri))


Service State: Healthy
Scoring URI: http://6809cf71-fe92-4844-8848-ea884b0a4286.southcentralus.azurecontainer.io/score
Swagger URI: http://6809cf71-fe92-4844-8848-ea884b0a4286.southcentralus.azurecontainer.io/swagger.json


In [108]:
import requests
import json

scoring_uri=service.scoring_uri

data = {'data':
            {
                "golddiffat10": -500,
                "xpdiffat10": -300,
                "golddiffat15": 2000,
                "xpdiffat15": 1600
            }
        }

input_data = json.dumps(data)

print(input_data)

response = service.run(input_data)
print(response)

{"data": {"golddiffat10": -500, "xpdiffat10": -300, "golddiffat15": 2000, "xpdiffat15": 1600}}


ERROR:azureml.core.webservice.aci:Received bad response from service. More information can be found by calling `.get_logs()` on the webservice object.
Response Code: 502
Headers: {'Connection': 'keep-alive', 'Content-Length': '50', 'Content-Type': 'text/html; charset=utf-8', 'Date': 'Tue, 26 Jan 2021 06:05:16 GMT', 'Server': 'nginx/1.10.3 (Ubuntu)', 'X-Ms-Request-Id': '3e2d4042-6212-4b53-8de6-da936f3abff3', 'X-Ms-Run-Function-Failed': 'True'}
Content: b'If using all scalar values, you must pass an index'



WebserviceException: WebserviceException:
	Message: Received bad response from service. More information can be found by calling `.get_logs()` on the webservice object.
Response Code: 502
Headers: {'Connection': 'keep-alive', 'Content-Length': '50', 'Content-Type': 'text/html; charset=utf-8', 'Date': 'Tue, 26 Jan 2021 06:05:16 GMT', 'Server': 'nginx/1.10.3 (Ubuntu)', 'X-Ms-Request-Id': '3e2d4042-6212-4b53-8de6-da936f3abff3', 'X-Ms-Run-Function-Failed': 'True'}
Content: b'If using all scalar values, you must pass an index'
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Received bad response from service. More information can be found by calling `.get_logs()` on the webservice object.\nResponse Code: 502\nHeaders: {'Connection': 'keep-alive', 'Content-Length': '50', 'Content-Type': 'text/html; charset=utf-8', 'Date': 'Tue, 26 Jan 2021 06:05:16 GMT', 'Server': 'nginx/1.10.3 (Ubuntu)', 'X-Ms-Request-Id': '3e2d4042-6212-4b53-8de6-da936f3abff3', 'X-Ms-Run-Function-

TODO: In the cell below, print the logs of the web service and delete the service

In [61]:
service = AciWebservice(name='lolpredictor', workspace=ws)
print(service.get_logs())

2021-01-26T04:28:36,245117717+00:00 - gunicorn/run 
2021-01-26T04:28:36,245481517+00:00 - iot-server/run 
2021-01-26T04:28:36,254430536+00:00 - rsyslog/run 
2021-01-26T04:28:36,263178555+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
rsyslogd

In [28]:
service.delete()

WebserviceException: WebserviceException:
	Message: There is a deployment operation in flight for the Service: predictor
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "There is a deployment operation in flight for the Service: predictor"
    }
}